Подключаем библиотеки

In [198]:

from flask import Flask, request, Response
from nornir import InitNornir
import pynetbox
import ipaddress
from jinja2 import Environment, FileSystemLoader
import json
import macaddress
from pydhcpdparser import parser

Инициализируем nornir

In [199]:
# Init Nornir
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.101:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

Инициализируем pynetbox

In [200]:
# Init pynetbox
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [201]:
def convert_to_str(value):
    return '' if value is None else str(value)

In [202]:
def configure_interface_ipv4_address(netbox_ip_address):
    """
    Извлечение IPv4 адреса, маски, сети, префикса, шлюза.
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :return: ip4_address
    """

    ip4_address = format(ipaddress.IPv4Interface(netbox_ip_address).ip)
    ip4_netmask = format(ipaddress.IPv4Interface(netbox_ip_address).netmask)
    ip4_network = format(ipaddress.IPv4Interface(netbox_ip_address).network)
    ip4_prefix = format(ipaddress.IPv4Network(ip4_network).prefixlen)
    ip4_broadcast = format(ipaddress.IPv4Network(ip4_network).broadcast_address)
    ip4_gateway = format(list(ipaddress.IPv4Network(ip4_network).hosts())[-1])
    #print(ip4_address,ip4_netmask, ip4_network, ip4_prefix, ip4_broadcast, ip4_gateway)

    return(ip4_address)

In [203]:
def confiugure_interface_mac_address(netbox_mac_address):
    #netbox_mac_address = '2A:2F:0A:9F:A8:B5'
    #netbox_mac_address = None

    netbox_mac_address = convert_to_str(netbox_mac_address)
    mac_address = netbox_mac_address
    hw_addr = macaddress
    try:
        hw_addr.MAC(mac_address)
    except ValueError as error:
        #print(error)
        mac_address = None
    print("MAC address is {}...".format(mac_address))
    return(mac_address)

#mac_address = None
#confiugure_interface_mac_address(mac_address)

In [204]:
def test_for_equal(mac_address):
    """
    Проверяем файл с конфигурацией на наличие в нем каких-либо настроек, связанных с передаваемым параметром
    :param device_name
    :return: None 
    """
    device_name = ''
    
    """
    Находим составное имя (device_name) по MAC адресу
    """
    hw_addr = mac_address.strip()
    with open("result.conf", "r") as f:
        conf = f.read()
        f.close()
    config = parser.parse(conf)
    dic_config = list(config[0]['host'])
    for dev_name in dic_config:
        if hw_addr in (s for s in config[0]['host'][dev_name]['hardware'].values()):
            print("Find {} for device {}".format(hw_addr, dev_name))
            device_name = dev_name

    """
    Если имя находим, то удалаяем все настройки, связанные с этим именем
    """
    start,end = -2,-2 # start и end должны быть не пустыми и меньше -1
    name = device_name.strip().casefold() # избавляемся от пробелов и игнорируем регистр
    with open("result.conf", 'r') as f: # открываем файл для чтения
        config = f.readlines() # считывем построчно и получаем список
    for line in config:
        if line.startswith('host'): # если в начале строки попадается 'host'
            if name in (s for s in line.strip().casefold().split()): # перебираем строку поэлементно на совпадение с 'name'
                start = config.index(line) # запоминаем индекс
                for i in range(start+1,len(config)): # продолжаем перебирать строки, начиная с индекса
                    if '}' in (s for s in list(config[i])): # разбиваем строку на отдельные символы и проверяем на совпадение с '}'
                        end = i
                        break
    
    if start >= 0 and end >=start : # значения для среза обязательно должны быть, и второе (end) должно быть не меньше первого (start)
        del config[start:end+1] # делаем срез списка (избавляемся от строк)
        with open("result.conf", 'w') as w:
            w.writelines(config)
            w.close()
        print("'{}' is find and deleted there config...".format(mac_address))
    else: print("Configuration file does not contain '{}'...".format(mac_address)) 

#test_for_equal('2A:2F:0A:9F:A8:B5')

In [205]:
def dhcpd_config_file(j2_ip_address,j2_interface,event='None'):
    
    """ 
    Заполнение шаблона значениями
    :param j2_device_name: устройство 
    :param j2_mac_address: mac адрес
    :param j2_ip_address: ip адрес
    :return: None
    """
    
    if confiugure_interface_mac_address(j2_interface.mac_address) != None:
    
        test_for_equal(j2_interface.mac_address)
        
        if event != 'delete':
            templates_path = "./templates/"
            environment = Environment(loader=FileSystemLoader(templates_path))
            template = environment.get_template("dhcpd_static.template")

            content = template.render(
            device_name = j2_interface.device.name+'.'+j2_interface.name.replace(" ","_"),
            host_name = j2_interface.device.name,
            mac_address = j2_interface.mac_address,
            ip_address = j2_ip_address
            )

            # Сохраним получившийся конфиг
            with open('result.conf', 'a') as fp: 
                fp.write(content + '\n')
                fp.close()
            print("File {} is saved...".format("result.conf"))

            """ # Прочитаем записанный файл
            with open('result.conf', 'r') as fp: 
                print(fp.read()) """
    
    else: print("MAC address isn’t compared...")
    
#dhcpd_config_file('SRV-PEV-1','None', '1.1.1.1', 'create')

In [206]:
def delete_ip_address(netbox_interface,netbox_ip_address,netbox_address_family):
    
    """  
    Удаление ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """
    
    print(f"Removing address {netbox_ip_address} "
          f"from interface '{netbox_interface.name}' "
          f"on device '{netbox_interface.device.name}'...")

    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        interface = netbox_interface
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    
    """
    Удаляем запись из конфигурационного файла DHCPd-службы
    """
    dhcpd_config_file(ip_address,interface,event='delete')

In [207]:
def create_ip_address(netbox_interface,netbox_ip_address,netbox_address_family):
    
    """  
    Создание ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """

    print(f"Assigning address {netbox_ip_address} "
          f"to interface '{netbox_interface.name}' "
          f"on device '{netbox_interface.device.name}'...")
    
    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        interface = netbox_interface
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    
    """
    Добавляем запись в конфигурационный файл DHCPd-службы
    """
    dhcpd_config_file(ip_address,interface,event='create')

In [208]:
def update_ip_address(netbox_interface,snapshot_json,netbox_ip_address,netbox_address_family):
    """  
    Изменение ip адреса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :param netbox_address_family: Версия IP (4|6)
    :return: None
    """

    print("Updating IP address...")
    #print(snapshot_json)
    if snapshot_json:
        try:
            old_interface_id = snapshot_json["prechange"]["assigned_object_id"]

            if old_interface_id != netbox_interface.id:
                # Старое назначение принадлежит другому интерфейсу. 
                # Изменение конфигурации перед настройкой нового устройства.
                old_interface_data = nb.dcim.interfaces.get(old_interface_id)
                if not old_interface_data.mgmt_only:
                    delete_ip_address(  netbox_interface,
                                        netbox_ip_address,
                                        netbox_address_family)
        except AttributeError:
            print("Address not previously assigned")
        except ValueError:
            print("Address not previously assigned")

    if netbox_address_family == 4:
        ip_address = configure_interface_ipv4_address(netbox_ip_address)
        interface = netbox_interface
        print("IP address V4...")
    else:
        print("IPv6")
        #configure_interface_ipv6_address(netbox_ip_address)
    
    """
    Изменяем запись в конфигурационном файле DHCPd-службы
    """
    dhcpd_config_file(ip_address,interface,event='update')

Создаем функцию для манипулирования IP адресами

In [209]:
def manage_interface_ip_address():
        
    #get_device = request.json["data"]["assigned_object"]["device"]["name"]
    get_device_interface = nb.dcim.interfaces.get(request.json["data"]["assigned_object_id"])    
    get_device_ips = request.json["data"]["address"]
    get_address_family = request.json["data"]["family"]["value"]

    if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсов
        print("\tManagement interface, no changes will be performed...")
    else:
            if request.json["event"] == "deleted": # IP адрес будет удален

                delete_ip_address(  netbox_interface=get_device_interface,
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family
                                    )

            elif request.json["event"] == "created": # IP адрес будет добавлен.

                create_ip_address(netbox_interface=get_device_interface,
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family)

            elif request.json["event"] == "updated":
                # Details of the IP have changed.  Determine if it was previously
                # assigned to a different device / interface. If so, unconfigure
                # that interface first and configure on the new one.
                update_ip_address(  netbox_interface=get_device_interface,
                                    snapshot_json=request.json.get("snapshots"),
                                    netbox_ip_address=get_device_ips,
                                    netbox_address_family=get_address_family
                                    )
    
    return Response(status=204)

Создаем экземпляр Flask

In [210]:
# Create a Flask instance
app = Flask(__name__)
app.add_url_rule("/api/fixed_ip",
                 methods=["POST"],
                 view_func=manage_interface_ip_address)

if __name__ == "__main__":
    # If this script is called from the command line, instruct Flask to enable
    # debugging for the app and listen on every IP address on the specified
    # port.
    #app.debug = True
    app.run(host="0.0.0.0", port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.68.5.5:8080
Press CTRL+C to quit
10.30.1.101 - - [12/Mar/2023 20:09:30] "POST /api/fixed_ip HTTP/1.1" 204 -


Updating IP address...
IP address V4...
MAC address is A1:B3:C4:C5:C5:C2...
Find A1:B3:C4:C5:C5:C2 for device SRV-PEV-1.Gig-E_4 
'A1:B3:C4:C5:C5:C2' is find and deleted there config...
File result.conf is saved...


10.30.1.101 - - [12/Mar/2023 20:16:38] "POST /api/fixed_ip HTTP/1.1" 204 -


Updating IP address...
IP address V4...
MAC address is None...
MAC address isn’t compared...


...

Assigning address 10.30.1.102/24 to interface 'Gig-E 3' on device 'SRV-PEV-5'...
MAC address is None...
MAC address isn’t compared...


10.30.1.101 - - [12/Mar/2023 20:21:13] "POST /api/fixed_ip HTTP/1.1" 204 -


Updating IP address...
IP address V4...
MAC address is A1:B3:C4:C5:C3:C2...
Configuration file does not contain 'A1:B3:C4:C5:C3:C2'...
File result.conf is saved...
Updating IP address...


10.30.1.101 - - [12/Mar/2023 20:22:47] "POST /api/fixed_ip HTTP/1.1" 204 -


Removing address 10.30.1.102/24 from interface 'Gig-E 4' on device 'SRV-PEV-5'...
MAC address is 01:23:43:44:55:43...
Configuration file does not contain '01:23:43:44:55:43'...
IP address V4...
MAC address is 01:23:43:44:55:43...
Configuration file does not contain '01:23:43:44:55:43'...
File result.conf is saved...
